## ITERATION
*python을 아름답게*

In [17]:
# python은 많은 것이 iterable합니다.

for element in [1, 2, 3]:
    print(element, end=' ')
for element in (1, 2, 3):
    print(element, end=' ')
for key in {'one':1, 'two':2}:
    print(key, end=' ')
for char in "123":
    print(char, end=' ')
for line in open("data/myfile.txt"):
    print(line, end='')

1 2 3 1 2 3 one two 1 2 3 total 3672
drwxr-xr-x@  4 happytk  staff      136 May 17 22:09 .
drwx------@ 49 happytk  staff     1666 May 17 22:08 ..
-rwxr-xr-x@  1 happytk  staff  1877810 May 12 18:46 WindowsUpdate.log
-rw-r--r--   1 happytk  staff        0 May 17 22:09 myfile.txt


Behind the scenes.
- forloop은 container object에 iter()함수를 호출합니다.
- return받은 객체의 __next__() 를 호출하면서 iterating할 대상을 하나씩 가져옵니다.
- StopIteration 오류가 발생하면 forloop을 멈추게 됩니다.

```
>>> s = 'abc'
>>> it = iter(s)
>>> it
<iterator object at 0x00A1DB50>
>>> next(it)
'a'
>>> next(it)
'b'
>>> next(it)
'c'
>>> next(it)
Traceback (most recent call last):
  File "<stdin>", line 1, in ?
    next(it)
StopIteration
```

In [18]:
# Iteration protocol을 구현해보기
class MyClassIterator:
    def __init__(self, *arg):
        self._data = arg
        self.index = 0
    def __next__(self):
        if len(self._data) == self.index:
            raise StopIteration
        ret = self._data[self.index]
        self.index += 1
        return ret

class MyClass:
    def __init__(self):
        self._data = [4,0,7,3]
    def __iter__(self):
        return MyClassIterator(*self._data)

for x in MyClass():
    print(x)

4
0
7
3


In [19]:
# 편의상(혹은 필요여부에 따라) 하나의 클래스에 구현하는 편입니다.
class MyClassSimple:
    def __init__(self):
        self._data = [4,0,7,3]
        self.index = 0
    def __iter__(self):
        return self
    def __next__(self):
        if len(self._data) == self.index:
            raise StopIteration
        ret = self._data[self.index]
        self.index += 1
        return ret    

for x in MyClassSimple():
    print(x)

4
0
7
3


In [20]:
from collections.abc import Iterable
(
    isinstance(MyClass(), Iterable),
    list(MyClass()),
    isinstance(MyClassSimple(), Iterable),
    list(MyClassSimple())
)

(True, [4, 0, 7, 3], True, [4, 0, 7, 3])

## GENERATOR (PEP 255)
*강력한 도구*

- iterator를 return하는 함수입니다.
- 일반 함수처럼 생겼는데 다만 함수 내 **yield** keyword가 사용됩니다.
- 일반 함수는 한번 수행되면 return까지 쉼 없이 수행되지만,
- generator는 yield를 만나면 잠시 멈추고 caller에게 수행권을 넘겨줍니다.
- 즉 (신기하게도) 함수가 **동작중 SUSPEND**될 수 있습니다.
- 다시 호출하면 SUSPEND되었던 부분에서 이어서 수행할 수 있습니다.

![](img/what-does-that-mean.gif)

확인해봅시다!

In [21]:
# 우리가 아는 일반함수
def well_known_function():
    return 10

well_known_function()

10

In [22]:
# yield가 사용되는 순간 함수가 아닌 generator로 변신.
def i_am_generator():
    print('funtion is started.')
    yield 10
    yield 20

# 수행 결과로 왠 객체가..
# g = i_am_generator()
# for x in g:, list(g), ....
for x in i_am_generator():
    print(x)

funtion is started.
10
20


In [23]:
def i_am_generator():
    print('funtion is started.')
    yield 10


# 해당객체는 iterable하므로 for문으로 꺼내볼 수 있습니다.
for x in i_am_generator():
    print(x)

funtion is started.
10


In [24]:
# 여러개의 yield문을 안에서 사용할 수 있습니다.
def yield_test():
    print('funtion is started.') # 이게 언제 실행되는지 보세요.
    yield 1
    yield 2
    yield 3
    print('function is finished.')
g = yield_test()
for x in g:
    print(x)

funtion is started.
1
2
3
function is finished.


In [25]:
from collections.abc import Iterable
gen = yield_test()

# iterable한 객체를 return하기 때문에 looping가능.
print(isinstance(gen, Iterable))

for x in yield_test():
    print(x)

True
funtion is started.
1
2
3
function is finished.


In [26]:
# iterator를 return하는
g = yield_test()
print('__iter__' in dir(g))
print('__next__' in dir(g))
next(g), next(g), next(g)

True
True
funtion is started.


(1, 2, 3)

In [27]:
# 함수 수행중 중단 및 재개
#sub
def yield_test():
    yield 1
    print('resumed')
    yield 2
    print('resumed')
    yield 3

#main
for x in yield_test():
    print(x)

1
resumed
2
resumed
3


In [28]:
# return을 만나거나 함수끝까지 가면 generator는 끝납니다.
def gen():
    yield 10
    yield 20
    return
    yield 30
for x in gen():
    print(x)

10
20


In [29]:
# 설계상으로도 그렇고, return문은 의미가 없을 가능성이 높고
# (최초의 설계된) generator는 결과값을 받을 수 없었습니다.
def gen():
    yield 10
    yield 20
    yield 30
    return x
for x in gen():
    print(x)
g = gen()
next(g), next(g), next(g), next(g) # exception을 잘 살펴보세요.

10
20
30


StopIteration: 30

In [ ]:
# 연습문제: 로또번호자동생성기
import random
# random.shuffle()

def lottery(numbers=45, count=6):
    # 0 - cout 사이의 숫자를 random하게 cout번 내어주도록
    numbers = list(range(1, numbers+1))
    random.shuffle(numbers)
    for number in numbers[:count]:
        yield number
#     return numbers[:count]

for random_number in lottery(45):
    print("And the next number is... %d!" %(random_number))

In [ ]:
# 연습문제: fibonacci
# fill in this function
def fib():
    pass #this is a null statement which does nothing when executed, useful as a placeholder.

# testing code
import types
if type(fib()) == types.GeneratorType:
    print("Good, The fib function is a generator.")

    counter = 0
    for n in fib():
        print(n)
        counter += 1
        if counter == 10:
            break

In [ ]:
def fib():
    val1, val2 = 1, 1
    while True:
        yield val1
        val1, val2 = val2, (val1+val2)

# testing code
import types
if type(fib()) == types.GeneratorType:
    print("Good, The fib function is a generator.")

    counter = 0
    for n in fib():
        print(n, end=', ')
        counter += 1
        if counter == 10:
            break

뭐가 특별한건가요? LIST와 무엇이 다른가요?
- 한번 소비되면 끝이에요.
- 데이터를 무한으로 생성 가능해요.
- LIST보다 메모리를 조금 써요.
- LIST사용하는 것보다는 조금 느릴 수 있어요.
- len이나 slice를 generator에는 사용할 수 없습니다.

python3은 laziness가 python3의 주요 테마이자 trend입니다.

```
# python2
range(10), xrange(10)
open(file).readlines, open(file).xreadlines
{}.keys()

# python3
range(10)
open(file).readlines()
{}.keys()
```

**<u>generator는 쉽게 laziness를 구현할 수 있습니다.</u>**

기본적으로 list를 return하는 함수를 generator로 바꿔서 사용할 수 있습니다.
- 한번 쓰고 재사용되지 않을 것이라면
- 메모리를 과점유하고 싶지 않다면
- database record fetching

In [ ]:
def counter_list(size):
    results = []
    cur = 1
    while cur <= size:
        results.append(cur)
        cur = cur + 1
    return results

def counter_gen(size):
    cur = 1
    while cur <= size:
        yield cur
        cur = cur + 1

list(counter_gen(500)) == counter_list(500)

## GENERATOR EXPRESSION (AGAIN)

In [ ]:
[str(num) for num in range(3)]

In [ ]:
(str(num) for num in range(3))

In [ ]:
# generator는 한번 사용하면 버려져요!
g = (str(num) for num in range(3))
list(g), list(g)

In [ ]:
# 함수의 단일인자로 사용할때는 ()가 필요없어집니다.
numbers = list(range(10))
(
    sum([x+2 for x in numbers]),
    sum((x+2 for x in numbers)),
    sum(x+2 for x in numbers),
    sum([x+2 for x in numbers], 5),
    sum((x+2 for x in numbers), 5),
#     sum(x+2 for x in numbers, 5)
)

## EXERCISE

In [ ]:
# 1. list를 인자로 받아 음수를 filtering하는 generator를 만들어보세요.
# 2. generator expression으로 바꿔보세요

# ```
# >> list(pos_gen(range(-10,10)))
# [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
# ```

def pos_gen(iterable):
    for value in iterable:
        if value >= 0:
            yield value

# list(pos_gen(range(-10,10)))
list(value for value in range(-10, 10) if value >= 0)

In [ ]:
# 정답
def pos_gen(seq):
    for x in seq:
        if x >= 0:
            yield x
list(pos_gen(range(-10,10)))

In [ ]:
def pos_gen(seq):
    return (x for x in seq if x >= 0)
list(pos_gen(range(-10,10)))

In [ ]:
pos_gen = lambda seq: filter(lambda x: x>=0, seq)
list(pos_gen(range(-10,10)))

## EXERCISE

영어문장을 받아서 각 단어의 앞글자만 대문자로 유지해주는 함수를 만들어봅시다. (문자열의 capitalize method를 사용하세요.)

```b
>> capwords(" abc  dEf ")
"Abc Def"
```


In [32]:
def capwords(string):
    return ' '.join(
        word.capitalize()
        for word in string.split()
    )

capwords(" asdf defaf")

'Asdf Defaf'

In [ ]:
# Lib/string.py 표준라이브러리에서 
# generator expression 사용된 것이 보이시나요?

# Capitalize the words in a string, e.g. " aBc  dEf " -> "Abc Def".
def capwords(s, sep=None):
    """capwords(s [,sep]) -> string

    Split the argument into words using split, capitalize each
    word using capitalize, and join the capitalized words using
    join.  If the optional second argument sep is absent or None,
    runs of whitespace characters are replaced by a single space
    and leading and trailing whitespace are removed, otherwise
    sep is used to split and join the words.

    """
    return (sep or ' ').join(x.capitalize() for x in s.split(sep))

*1천억건의 데이터를 가진 list를 생성해서 사용하고자 한다. 한개 써먹었는데 갑자기 일이 생겨서(문제가 생겨서) 그만둬야겠다.*
![](https://trustliveserve.files.wordpress.com/2013/06/heavy-burden.png)

In [ ]:
list(range(1_000_000_000_000))

In [ ]:
r = range(1_000_000_000_000)
i = iter(r)
next(i)
next(i)

In [39]:
def myfunc():
    yield "12345"
    yield "45678"
    yield "90123"

# 파일을 읽고 싶은만큼만 읽으려면?
# for line in open('data/myfile.txt').readlines()[:3]:
# for line in myfunc()[:1]:
#    print(line)

from itertools import islice
for line in islice(open('data/myfile.txt'), 3):
    print(line, end='')

total 3672
drwxr-xr-x@  4 happytk  staff      136 May 17 22:09 .
drwx------@ 49 happytk  staff     1666 May 17 22:08 ..


내가 요구할때 <span class="burk">하나씩</span> 주시면 됩니다.

- 메모리를 최대한 적게 사용하고
- 가능한 처리를 **늦게까지** 미루는

<span class="mark">PYTHON에서는 많은 것들이 iterable하고, 사용자도 쉽게 iterable한 데이터를 만들 수 있습니다.</span>

<span class="mark">PYTHON에서는 많은 것들이 iterable하고 lazy하며, 사용자도 쉽게 iterable하고 lazy한 데이터를 만들 수 있습니다.</span>

## itertools

In [ ]:
from itertools import (
    count,
    cycle,
    repeat,
    chain,
    accumulate,
    dropwhile,
    groupby,
    islice,
    tee,
    takewhile,
    zip_longest,
)

설명은 도움말을 참고해보아요
https://docs.python.org/3.6/library/itertools.html#module-itertools

## yield from 
pep 380: syntax for delegating to a subgenerator

여러개의 generator를 하나로 묶어주고 싶을 때

In [40]:
def gen_3seq(x):
    yield x+1
    yield x+2
    yield x+3

def gen_6seq(x):
    yield gen_3seq(x)
    yield gen_3seq(x+3)
    
for x in gen_6seq(0):    
    print(x)

<generator object gen_3seq at 0x1043ee410>
<generator object gen_3seq at 0x1043eee08>


In [41]:
# 안에서 한번 더 풀어줘야 합니다.
def gen_6seq(x):
    for val in gen_3seq(x):
        yield val
    for val in gen_3seq(x+3):
        yield val

list(gen_6seq(0))

[1, 2, 3, 4, 5, 6]

In [ ]:
# better way, python3.3부터 사용가능합니다.
def gen_6seq(x):
    yield from gen_3seq(x)
    yield from gen_3seq(x+3)

list(gen_6seq(0))

In [42]:
from itertools import chain
list(chain(gen_3seq(0), gen_3seq(3)))
# 이것도 좋은 방법이지만 개별로 예외처리가 어려워요.

[1, 2, 3, 4, 5, 6]

In [43]:
def gen_3seq(x):
    for v in range(x, x+3):
        if v == 2:
            raise ValueError('3 is not allowed')
        yield v

from itertools import chain
list(chain(gen_3seq(0), gen_3seq(3)))

ValueError: 3 is not allowed

In [44]:
def gen_3seq(x):
    for v in range(x, x+3):
        if v == 2:
            raise ValueError('3 is not allowed')
        yield v

def gen_6seq(x):
    try:
        yield from gen_3seq(x)
    except ValueError:
        yield 'error'
    yield from gen_3seq(x+3)

list(gen_6seq(0))

[0, 1, 'error', 3, 4, 5]

yield from은 이 외에도 복잡한 문제들을 다수 해결하기 위해서 등장했지만, 여기서는 간단한 syntax-sugar로만 일단 다루고 넘어갑니다. 물론 예외처리도 중요한 해결하고자 하는 문제 중 하나였습니다.

## COROUTINE (PEP 492) with asyncio (PEP3153)

PYTHON3에서 제공하는 ASYNCIO를 이용해서 COROUTINE을 쉽게 생성할 수 있습니다.

In [ ]:
import asyncio

In [ ]:
async def greeting():
    print("Hello World")

greeting()

In [ ]:
loop = asyncio.get_event_loop()
loop.run_until_complete(greeting())
loop.close()

In [ ]:
async def compute(future):
    print('starting...')
    res = await answer()
    future.set_result(res)
async def answer():
    await asyncio.sleep(1)
    return 42

f = asyncio.Future()
loop = asyncio.get_event_loop()
loop.run_until_complete(compute(f))
loop.close()
f.result()